# XGBoost

In [ ]:
%pip install -q pandas matplotlib numpy scikit-learn ace_tools panelsplit shap upgrade jupyter ipywidgets xgboost
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.



In [4]:
import xgboost as xgb
import torch
import shap
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

import zipfile
import os

from panelsplit.cross_validation import PanelSplit
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

c:\Users\cpedr\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Define file names
csv_filename = "../final_df.csv"
zip_filename = "../final_df.zip"

# Extract the CSV file from the ZIP
with zipfile.ZipFile(zip_filename, 'r') as zipf:
    zipf.extract(csv_filename)

# Read the CSV file into a DataFrame
final_df = pd.read_csv("final_df.csv")

# Delete the extracted CSV file after reading
os.remove("final_df.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../final_df.zip'

In [ ]:
# Data after 2007
final_df = final_df[final_df['year'] > 2006]
# y 
y = final_df['tc_loss']

# Normalize output
scaler = StandardScaler()
y = scaler.fit_transform(y.values.reshape(-1, 1))

# Get rid of columns that start with 'subnational1_' and 'cluster_' in train and test   
#X1 = final_df.loc[:,~final_df.columns.str.startswith('g')]
X1 = final_df.loc[:,~final_df.columns.str.startswith('subnational1_')]
X1 = X1.loc[:,~X1.columns.str.startswith('cluster_')]


# Get rid of all disaggregated columns
X1 = X1.loc[:,~X1.columns.str.startswith('ac_')]
X1 = X1.loc[:,~X1.columns.str.startswith('as_')]
X1 = X1.loc[:,~X1.columns.str.startswith('p_')]
X1 = X1.loc[:,~X1.columns.str.startswith('r_')]
X1 = X1.loc[:,~X1.columns.str.startswith('nuf_')]
X1 = X1.loc[:,~X1.columns.str.startswith('vrf_')]

# X final
X1 = X1.drop(columns=['year', 'tc_loss', 'tc_loss_area', 'codmpio'])

# Keep feature names
original_feature_names = list(X1.columns) 

# Create polynomial interaction terms (degree=2, only interactions, no bias term)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions = poly.fit_transform(X1)

# Get the feature names including interactions
interaction_feature_names = poly.get_feature_names_out(original_feature_names)

# Compute standard deviation of each feature
stds = X1.std(axis=0)

# Find columns where std = 0
zero_variance_features = np.where(stds == 0)[0]

if len(zero_variance_features) > 0:
    print(f"⚠️ Removing {len(zero_variance_features)} features with zero variance.")
    print(f"🔍 Removed feature indices: {zero_variance_features}")

    # If feature names are available, print them
    if isinstance(X1, pd.DataFrame):  # If X is a DataFrame
        removed_feature_names = X1.columns[zero_variance_features]
        print(f"📌 Removed feature names: {list(removed_feature_names)}")

    X1 = np.delete(X1, zero_variance_features, axis=1)  # Remove constant columns


# Normalize features
X1 = scaler.fit_transform(X1)

In [ ]:


# 🔹 Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🔹 Define XGBoost Model Parameters
xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "max_depth": 6,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "alpha": 0.01,  # L1 Regularization
    "lambda": 0.1,  # L2 Regularization
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

panel_split = PanelSplit(periods=final_df.year, n_splits=5)

# 🔹 Store results for cross-validation
cv_results = []

for train_idx, test_idx in panel_split.split(X1):

    # Create interaction terms (degree=2 means pairwise interactions)
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_interactions = poly.fit_transform(X1)
    
    # Split data
    X_train, X_test = X1[train_idx], X1[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Convert to PyTorch tensors (for consistency)
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    # 🔹 Train XGBoost Model
    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_train_tensor.cpu().numpy(), y_train_tensor.cpu().numpy(),
                  eval_set=[(X_test_tensor.cpu().numpy(), y_test_tensor.cpu().numpy())],
                  early_stopping_rounds=50, verbose=False)

    # 🔹 Get Predictions
    y_train_pred = xgb_model.predict(X_train_tensor.cpu().numpy())
    y_test_pred = xgb_model.predict(X_test_tensor.cpu().numpy())

    # 🔹 Compute Performance Metrics
    mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_test_pred)
    rmse = np.sqrt(mse)
    r2_test = r2_score(y_test_tensor.cpu().numpy(), y_test_pred)
    r2_train = r2_score(y_train_tensor.cpu().numpy(), y_train_pred)

    # 🔹 Store Results
    cv_results.append({"MSE": mse, "RMSE": rmse, "R2_train": r2_train, "R2_test": r2_test})

# 🔹 Compute Averages Across CV Splits
avg_results = {metric: np.mean([fold[metric] for fold in cv_results]) for metric in cv_results[0].keys()}

# 🔹 Print Results
print(f"✅ XGBoost with TimeSeriesSplit Results:")
print(f"📊 Avg MSE: {avg_results['MSE']:.4f}")
print(f"📊 Avg RMSE: {avg_results['RMSE']:.4f}")
print(f"📊 Avg R² (Train): {avg_results['R2_train']:.4f}")
print(f"📊 Avg R² (Test): {avg_results['R2_test']:.4f}")

# 🔹 Compute SHAP Values for Feature Importance
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test_tensor.cpu().numpy())

# 🔹 Plot SHAP Feature Importance
shap.plots.bar(shap_values)
